In [1]:
import pymorphy2
import nltk
import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
import gzip

from datetime import datetime

from collections import Counter

In [2]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res

In [3]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 10):
    data = json.loads(line.strip())
    print(Document(data))

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_mai.jsonl.gz'

In [4]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))
    
for line in itertools.islice(fin, 5000):
    data = json.loads(line.strip())
    dataset_test.append(Document(data))

In [5]:
test_doc = dataset[11]
print(test_doc)

url : https://trikky.ru/kakoy-ty-napitok-423256.html
date : 2019-11-30 18:26:48
title : Какой ты напиток?
description : Какой ты напиток?Зелёненький,свеженький мохито,или взрывной лимонад?А может быть,Кока Кола?Сейчас  и узнаем!1. Какой вкус в напитках тебе больше всего нравится? Яркие цвета - вот что самое главное! Эта многослойность напитков заставляет завораживаться... Сладость,я такая сладкоежка! Я обожаю газировку,такая взрывная,как шипучка!2. Какие типы цветов напитков тебе больше нравятся? Холодные(голубые,синие,фиолетовые разных оттенков) Чёрный/Белый/Белый в крапинку Зелёные,цвет природы. Яркие […]
site : trikky.ru



### Как поделить текст на слова?

In [6]:
def print_words(a_document, a_split_func):
    print('------TITLE----------')
    for word in a_split_func(test_doc.title):
        print(word)

    print('------DESCRIPTION----------')
    for word in a_split_func(test_doc.description):
        print(word)

In [7]:
### Побить по пробелам?
def split_words_v1(a_text):
    for word in a_text.split():
        yield word

print_words(test_doc, split_words_v1)

------TITLE----------
Какой
ты
напиток?
------DESCRIPTION----------
Какой
ты
напиток?Зелёненький,свеженький
мохито,или
взрывной
лимонад?А
может
быть,Кока
Кола?Сейчас
и
узнаем!1.
Какой
вкус
в
напитках
тебе
больше
всего
нравится?
Яркие
цвета
-
вот
что
самое
главное!
Эта
многослойность
напитков
заставляет
завораживаться...
Сладость,я
такая
сладкоежка!
Я
обожаю
газировку,такая
взрывная,как
шипучка!2.
Какие
типы
цветов
напитков
тебе
больше
нравятся?
Холодные(голубые,синие,фиолетовые
разных
оттенков)
Чёрный/Белый/Белый
в
крапинку
Зелёные,цвет
природы.
Яркие
[…]


In [8]:
### Побить по всем не буквам и цифрам?
def split_words_v2(a_text):
    cur_word = ''
    for letter in a_text:
        if letter.isalnum():
            cur_word += letter
        else:
            if cur_word: yield cur_word
            cur_word = ''
            
    if cur_word: yield cur_word
    
print_words(test_doc, split_words_v2)

------TITLE----------
Какой
ты
напиток
------DESCRIPTION----------
Какой
ты
напиток
Зелёненький
свеженький
мохито
или
взрывной
лимонад
А
может
быть
Кока
Кола
Сейчас
и
узнаем
1
Какой
вкус
в
напитках
тебе
больше
всего
нравится
Яркие
цвета
вот
что
самое
главное
Эта
многослойность
напитков
заставляет
завораживаться
Сладость
я
такая
сладкоежка
Я
обожаю
газировку
такая
взрывная
как
шипучка
2
Какие
типы
цветов
напитков
тебе
больше
нравятся
Холодные
голубые
синие
фиолетовые
разных
оттенков
Чёрный
Белый
Белый
в
крапинку
Зелёные
цвет
природы
Яркие


In [9]:
### Что делать с разными написаниями iphone6 и iphone 6 ? Как сделать чтобы всегда получать одну версию разбиения?
## Можно оставить студентам написать?
test_text = 'Где купить iPhone 6? и почему iPhone6 стоит так дорого?'
print(list(split_words_v1(test_text)))
print(list(split_words_v2(test_text)))

['Где', 'купить', 'iPhone', '6?', 'и', 'почему', 'iPhone6', 'стоит', 'так', 'дорого?']
['Где', 'купить', 'iPhone', '6', 'и', 'почему', 'iPhone6', 'стоит', 'так', 'дорого']


## Реализовать функцию которая единообразно будет разбивать такие строки?

In [10]:
def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word
         
print(list(split_words_v3(test_text)))

['Где', 'купить', 'iPhone', '6', 'и', 'почему', 'iPhone', '6', 'стоит', 'так', 'дорого']


In [11]:
words = list(split_words_v2(test_doc.title + ' ' + test_doc.description))

In [12]:
len(words)

74

In [13]:
uniq_words = Counter(words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))

NumKeys :  63
[('Какой', 3), ('ты', 2), ('напиток', 2), ('в', 2), ('тебе', 2), ('больше', 2), ('Яркие', 2), ('напитков', 2), ('такая', 2), ('Белый', 2)]


## Могут ли быть разные написания одного слова?

In [14]:
def get_norm_word_v1(a_word):
    return a_word.lower()

uniq_words = Counter(get_norm_word_v1(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('я', 2)

NumKeys :  62
[('какой', 3), ('ты', 2), ('напиток', 2), ('в', 2), ('тебе', 2), ('больше', 2), ('яркие', 2), ('напитков', 2), ('я', 2), ('такая', 2)]


In [15]:
from nltk.stem.snowball import RussianStemmer

In [16]:
stemmer = RussianStemmer()

In [17]:
stemmer.stem('Напитков')

'напитк'

In [18]:
def get_norm_word_v2(a_word):
    return stemmer.stem(a_word)

In [19]:
uniq_words = Counter(get_norm_word_v2(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('я', 2)

NumKeys :  55
[('как', 5), ('напитк', 3), ('цвет', 3), ('ты', 2), ('напиток', 2), ('взрывн', 2), ('в', 2), ('теб', 2), ('больш', 2), ('нрав', 2)]


In [20]:
for word in words:
    if get_norm_word_v2(word) == 'как':
        print(word)

Какой
Какой
Какой
как
Какие


In [21]:
for word in words:
    if get_norm_word_v2(word) == 'цвет':
        print(word)

цвета
цветов
цвет


In [22]:
morth_analyzer = pymorphy2.MorphAnalyzer()

In [23]:
res = morth_analyzer.parse('Какая') #Какая , Напитков

In [24]:
res

[Parse(word='какая', tag=OpencorporaTag('ADJF,Apro femn,sing,nomn'), normal_form='какой', score=0.987012, methods_stack=((<DictionaryAnalyzer>, 'какая', 1676, 7),)),
 Parse(word='какая', tag=OpencorporaTag('GRND,impf,intr pres'), normal_form='какать', score=0.012987, methods_stack=((<DictionaryAnalyzer>, 'какая', 15, 67),))]

In [25]:
res[0].normal_form

'какой'

In [26]:
MORTH_CACHE = {}
def get_norm_word_v3(a_word):
    if a_word not in MORTH_CACHE: MORTH_CACHE[a_word] = morth_analyzer.parse(a_word)[0].normal_form
    return MORTH_CACHE[a_word]

In [27]:
uniq_words = Counter(get_norm_word_v3(word) for word in words)
print("NumKeys : ", len(uniq_words))
print(uniq_words.most_common(10))
### ('напиток', 2)

NumKeys :  56
[('напиток', 5), ('какой', 4), ('ты', 4), ('взрывной', 2), ('в', 2), ('большой', 2), ('нравиться', 2), ('яркий', 2), ('я', 2), ('такой', 2)]


In [ ]:
### v4 - числа преоразовывать к специальному формату YYYY , DD - чтобы уменьшить словарь и склеивались похожие названия отличающиеся только цифрами 

### Пример деревьев решений для предсказания сайта по tf-idf векторам
Получится ли обучить хорошее решение линейной моделью?

In [28]:
def get_doc_words(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v3):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield a_norm_word(word)

In [29]:
class NaiveBayes:
    
    min_class_c = 12
    
    def __init__(self):
        self.word_in_class = Counter()
        self.word_not_in_class = Counter()
        
    def train(self, a_words, a_is_class):
        word_counter = self.word_in_class if a_is_class else self.word_not_in_class
        word_counter.update(a_words)
        
    def predict(self, a_words):
        res = 0
        num_words = 0
        for word in a_words:
            class_freq, not_class_freq = self.word_in_class.get(word, 0), self.word_not_in_class.get(word, 0)
            if class_freq < self.min_class_c or not_class_freq < self.min_class_c: continue
            num_words += 1
            res += np.log(class_freq/not_class_freq)
        return res/num_words if num_words > 0 else 0.0

In [30]:
dataset_sites = Counter(doc.site for doc in dataset)

sport_sites = set(['matchtv.ru', 'euro-football.ru', 'euro.com.ua',
                   'bobsoccer.ru', 'bombardir.ru', 'championat.com',
                   'football24.ru', 'rusfootball.info', 'soccernews.ru'
                  ])

for item in dataset_sites:
    if 'sport' in item: sport_sites.add(item)

In [31]:
for item in dataset_sites.most_common(1000):
    if item[0] not in sport_sites:
        print(item)

('tass.ru', 178)
('vk.com', 163)
('123ru.net', 143)
('gazeta.ru', 136)
('rossaprimavera.ru', 130)
('russian.rt.com', 118)
('daily-inform.ru', 108)
('newsyou.info', 95)
('365news.biz', 95)
('obozrevatel.com', 92)
('tvzvezda.ru', 83)
('moika78.ru', 78)
('life.ru', 73)
('wi-fi.ru', 71)
('riafan.ru', 71)
('rueconomics.ru', 69)
('pikabu.ru', 65)
('novosti-n.org', 63)
('mob-mobile.ru', 63)
('sm-news.ru', 58)
('news.ru', 58)
('mir24.tv', 57)
('nation-news.ru', 57)
('govoritmoskva.ru', 56)
('360tv.ru', 56)
('news-front.info', 53)
('terrikon.com', 53)
('vz.ru', 51)
('aif.ru', 51)
('dizainvfoto.ru', 50)
('argumenti.ru', 49)
('peterburg2.ru', 48)
('ukraina.ru', 46)
('oane.ws', 46)
('rosbalt.ru', 44)
('news.rambler.ru', 43)
('zelenj.ru', 40)
('ferra.ru', 39)
('bmwclub.ru', 39)
('habr.com', 39)
('deita.ru', 38)
('vestikavkaza.ru', 37)
('abnews.ru', 36)
('1rre.ru', 35)
('gazeta.spb.ru', 34)
('profile.ru', 34)
('fontanka.ru', 34)
('yerkramas.org', 33)
('riamo.ru', 33)
('radiosputnik.ria.ru', 32)
('ra

('dnevniki.ykt.ru', 2)
('yuripasholok.livejournal.com', 2)
('proseverouralsk.ru', 2)
('historygreatrussia.ru', 2)
('popcornnews.ru', 2)
('tobegirl.ru', 2)
('brodude.ru', 2)
('teknoblog.ru', 2)
('mycary.ru', 2)
('software-expert.ru', 2)
('jvlife.ru', 2)
('rb.ru', 2)
('realty.ria.ru', 2)
('vmolo.by', 2)
('rubankom.com', 2)
('ngs24.ru', 2)
('theblueprint.ru', 2)
('military.pravda.ru', 2)
('allfest.ru', 2)
('finrussia.ru', 2)
('tayga.info', 2)
('youloveit.ru', 2)
('fanera-info.ru', 2)
('spbvedomosti.ru', 2)
('pasmi.ru', 2)
('chips-journal.ru', 2)
('journal.tinkoff.ru', 2)
('7ya.ru', 2)
('rap.ru', 2)
('ochkov.net', 2)
('daily-motor.ru', 2)
('coinspot.io', 2)
('servernews.ru', 2)
('militaryexp.com', 2)
('ivday.ru', 2)
('paparazzi.ru', 2)
('dnpmag.com', 2)
('kinoart.ru', 2)
('gmbox.ru', 1)
('ura-povara.ru', 1)
('pogoda.mail.ru', 1)
('zapad24.ru', 1)
('3ddd.ru', 1)
('virtus.pro', 1)
('panorama.pravo.ru', 1)
('uchportal.ru', 1)
('agrognom.ru', 1)
('itzine.ru', 1)
('fitnessera.ru', 1)
('ladyspec

In [32]:
sport_dataset = [item for item in dataset if item.site in sport_sites]
not_sport_dataset = [item for item in dataset if item.site not in sport_sites]
print(len(sport_dataset), len(not_sport_dataset))

1087 8913


In [33]:
%%time

bayes_classifier = NaiveBayes()
for item in sport_dataset:
    bayes_classifier.train(get_doc_words(item), a_is_class=True)
    
for item in not_sport_dataset:
    bayes_classifier.train(get_doc_words(item), a_is_class=False)

CPU times: user 1min 9s, sys: 83.2 ms, total: 1min 9s
Wall time: 1min 9s


In [34]:
for item in not_sport_dataset:
    predict = bayes_classifier.predict(get_doc_words(item))
    if predict > np.log(len(sport_dataset)/len(not_sport_dataset)):
        print(predict, item)
#         break
        print('-*-'*13)


-1.9601938853085725 url : https://rusturkey.com/post/207586/grazhdanka-rossii-vypala-s-balkona-7-go-etazha-v-antale
date : 2019-11-30 18:26:11
title : Гражданка России выпала с балкона 7-го этажа в Анталье
description : Жизнь женщины не удалось спасти
site : rusturkey.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.837474331018305 url : https://riavrn.ru/news/voronezhskiy-batutist-stal-prizerom-chempionata-mira/
date : 2019-11-30 18:26:38
title : Воронежский батутист стал призером чемпионата мира
description : Сергей Азарян отличился в паре с Михаилом Мельником из Тольятти.
site : riavrn.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0732243592395294 url : https://tengrinews.kz/events/krupnyiy-pojar-ohvatil-bannyiy-kompleks-u-podnojiya-kok-tobe-385419/
date : 2019-11-30 18:22:16
title : Крупный пожар охватил банный комплекс у подножия Кок-Тобе
description : Крупный пожар охватил банный комплекс "Царское село", расположенный у подножия горы Кок-Тобе в Алматы вечером в субботу, 30 ноября

-0.9765743279060152 url : https://properm.ru/news/sport/177675/
date : 2019-11-30 18:31:17
title : «Молот-Прикамье» обыграл китайский клуб «КРС-БСУ»>
description : Следующий матч пермская команда вновь проведет против соперника из Китая.
site : properm.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.5218253598732354 url : https://russian.rt.com/sport/article/692025-biatlon-singl-mikst-estersund
date : 2019-11-30 18:31:44
title : Российские биатлонисты заняли 12-е место в сингл-миксте на этапе КМ в Эстерсунде
description : В шведском Эстерсунде стартовал новый розыгрыш Кубка мира по биатлону. Золото в сингл-миксте завоевали участники сборной страны — хозяйки соревнований Ханна Эберг и Себастьян Самуэльссон. Серебро досталось немцам Франциске Пройсс и Эрику Лессеру, бронза — норвежцам Марте Ройселанн и Ветле Шосте Критиансену. Россияне Тамара Воронина и Матвей Елисеев финишировали на 12-м месте. RT вёл текстовую онлайн-трансляцию гонки.
site : russian.rt.com

-*--*--*--*--*--*--*--*--*--*

-1.9123874570166697 url : https://pikabu.ru/story/meow_7075819
date : 2019-11-30 18:37:11
title : Meow?
description : https://www.instagram.com/oheekolts/ https://www.artstation.
site : pikabu.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9878358654999784 url : https://nsn.fm/in-the-world/ssha-nanesli-vozdushnyi-udar-po-obektam-terroristov-na-uge-somali
date : 2019-11-30 18:37:17
title : США нанесли воздушный удар по объектам террористов на юге Сомали
description : Вооруженные силы США нaнесли aвиaудaр по целям террористической оргaнизaции «Аш-Шабаб» нa юге Сомали
site : nsn.fm

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-0.9797901068686389 url : http://terrikon.com/posts/341363
date : 2019-11-30 18:38:39
title : Биатлон: украинцы - четвертые в одиночной смешанной эстафете
description : Украинские биатлонисты были в шаге от пьедестала в стартовой гонке первого этапа Кубка мира
site : terrikon.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.0066501764583633 url : https://news.ru/autosport

-1.4753879386932516 url : https://www.f1-world.ru/news/news.php3?idnews=1911301825
date : 2019-11-30 18:46:13
title : Даниилу Квяту не понравилось поведение машины в квалификации
description : По итогам квалификации в Абу-Даби Даниил Квят оказался на 14-й позиции. В гонке россиянин будет стартовать 13-м...
site : f1-world.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9054013391730793 url : https://bazaar.ru/heroes/parties/yasmina-muratovich-i-mariya-minogarova-na-otkrytii-magazina-ttswtrs/
date : 2019-11-30 18:46:13
title : Ясмина Муратович и Мария Миногарова на открытии магазина TTSWTRS
description : Гости вечеринки на Патриарших прудах
site : bazaar.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.203889171370507 url : http://terrikon.com/posts/341360
date : 2019-11-30 18:47:40
title : Олимпик несет кадровые потери перед матчем с Динамо
description : Последний день осени ознаменовался для команды Висенте Гомеса окончанием подготовки к матчу против "Динамо"
site : terrikon.com

-*--*--

-1.5360976353737685 url : https://www.f1news.ru/news/f1-141430.html
date : 2019-11-30 18:56:40
title : Танабе: В Toro Rosso смогут выбрать шины для старта
description : Пьер Гасли и Даниил Квят рассчитывают побороться за очки в Абу-Даби...
site : f1news.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.882672555323559 url : https://gazeta.spb.ru/2192257-peterburgskij-prazdnik-alye-parusa-priznali-luchshim-prazdnikom-v-mire/
date : 2019-11-30 18:56:12
title : Петербургский праздник «Алые паруса» признали лучшим праздником в мире
description : Главный праздник всех выпускников «Алые паруса», который проводится в Петербурге каждый год, получил главную награду в конкурсе Best Event Awards World. Всего было в Best Event Awards World 23 номинации. Петербургский праздник также наградили в номинациях «Лучшее музыкальное событие» и «Лучшее агентство в сфере организации праздников». Номинантов оценивали по креативности, наличию технологических инноваций, исполнению, общей […]
site : gazeta.spb.ru



...

-1.9012149638263875 url : https://vringe.com/news/129967-dyablo-i-cherkashin-vzvesilis-rezultaty-video-i-foto.htm
date : 2019-11-30 10:46:27
title : «Дьябло» и Черкашин взвесились: результаты, видео и фото
description : Состоялась процедура официального предматчевого взвешивания главных участников предстоящего вечера бокса на арене Nosalowy Dwór в Закопане (Польша). Боксёры показали следующий вес: Кшиштоф Влодарчик (57-4-1, 39 KO) — 90,8 кг Тейлор Мабика (19-4-1, 10 КО) — 87,4 кг 10-раундовый бой в тяжёлом весе (до 90,7 кг)  Фёдор Черкашин (14-0, 9 KO) — 73,7 кг Матиас Эклунд (11-2-2, 4 КО) —&nbs...
site : vringe.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.7606633577868833 url : https://sterlegrad.ru/newsrb/118110-respublikanskiy-festival-konkurs-bashkirskaya-shal-proydet-v-ufe.html
date : 2019-11-30 10:46:26
title : Республиканский фестиваль-конкурс «Башкирская шаль» пройдёт в Уфе
description : С 5 по 8 декабря в Уфе в рамках выставки-фестиваля народных художественных промыслов «У

0.0 url : https://www.gametech.ru/news/67303/
date : 2019-11-30 10:56:09
title : Релизный трейлер пошаговой стратегии Phoenix Point
description : 
site : gametech.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.5822488352427617 url : http://www.orenday.ru/novosti/novostnaya-lenta/301119123243
date : 2019-11-30 10:51:07
title : Оренбуржцев приглашают сыграть в "архивный квиз"
description : Любители интеллектуальных игр могут сразиться за звание лучшего знатока истории Оренбуржья.
site : orenday.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0612608755836543 url : https://riavrn.ru/news/v-voronezhskoy-oblasti-desyatka-vrezalas-v-derevo-pogib-33-letniy-voditel/
date : 2019-11-30 10:54:42
title : В Воронежской области «десятка» врезалась в дерево: погиб 33-летний водитель
description : ДТП произошло на автодороге М-4 «Дон» – Вишневый – Дубовиково.
site : riavrn.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.2760534828025667 url : https://24molnia.com/bez-rubriki/emelyanenko-otpravil-v-nokau

-2.0994931103116032 url : https://tarantas.news/posts/id6899-0uzudr03anljpk05mzec
date : 2019-11-30 10:57:09
title : Все новые Jaguar Land Rover будут обновлять «по воздуху»
description : Абсолютно все новые модели концерна Jaguar Land Rover получат возможность обновления прошивки комплекса мультимедиа «по воздуху». Расширенная функциональность будет доступна покупателям авто с системами Touch Pro и Touch Pro Duo, а владельцам старых транспортных средств предложат установить модернизированное оборудование на специальных условиях.
site : tarantas.news

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9361506044568033 url : https://lt.sputniknews.ru/incidents/20191130/10775852/V-Yonavskom-rayone-muzhchinu-na-stroyke-zasypalo-zemley.html
date : 2019-11-30 10:55:15
title : В Йонавском районе мужчину на стройке засыпало землей
description : Когда на место происшествия приехала пожарно-спасательная команда, над горой земли виднелась только голова строителя
site : lt.sputniknews.ru

-*--*--*--*--*-

-2.0603851718618995 url : https://vesti.kz/khl/273387/
date : 2019-11-30 11:01:37
title : "Барыс" объявил о расторжении контракта с легионером
description : Ранее он был выставлен на драфт отказов.
site : vesti.kz

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.3068180249526604 url : https://ruinformer.com/page/sanjok-ottjagival-nokaut-kak-mog-video
date : 2019-11-30 11:01:52
title : «Санёк оттягивал нокаут как мог» (ВИДЕО)
description : Александр Емельяненко отправил в нокаут пауэрлифтера Михаила Кокляева в первом раунде.
site : ruinformer.com

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9409262065582151 url : https://www.soccer.ru/blogs/record/1152572/cherchesov-stanislav-rossiya
date : 2019-11-30 11:03:06
title : Жеребьевка Евро-2020. Когда смотреть и чего ждать?
description : Финляндия или Уэльс – выбор для России, но «Соккер.ру» рассматривает весь расклад, ведь могут собрать интересные группы.
site : soccer.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-0.8359148011392309 url : https://w

-1.8889288848272616 url : https://vk-smi.ru/obshhestvo/166399
date : 2019-11-30 11:11:42
title : Москвичи обсудили развитие спорта на краудфандинг-платформе мэрии
description : 18000 москвичей участвовали в обсуждении проекта "Город спорта" на сайте мэрии 20-29 ноября. Пользователи предложили около 2000 идей по развитию физкультуры,
site : vk-smi.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.5165147763252527 url : https://privet-rostov.ru/rus/47836-oskorbljat-policejskih-v-seti-stanet-nakladno.html
date : 2019-11-30 11:11:43
title : Оскорблять полицейских в сети станет накладно
description : Оскорблять полицейских в сети станет накладно
site : privet-rostov.ru

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-1.9827479790493305 url : https://vesti.kz/uefa/273385/
date : 2019-11-30 11:11:44
title : Матч с "МЮ" стал юбилейным для двух футболистов "Астаны" в еврокубках
description : Один из них является самым опытным в клубе.
site : vesti.kz

-*--*--*--*--*--*--*--*--*--*--*--*--*-
-2.0921915335

In [35]:
### Попробывать обучить random_forest

In [36]:
%%time
word2id = {}
for item in dataset:
    item.words = list(get_doc_words(item))
    for word in item.words:
        if word not in word2id: word2id[word] = len(word2id)

CPU times: user 1min 7s, sys: 102 ms, total: 1min 7s
Wall time: 1min 7s


In [37]:
id2word = dict( (v,k) for k,v in word2id.items())

In [38]:
len(word2id)

31067

In [39]:
def doc2vector(a_doc):
    res = [0] * len(word2id)
    if not hasattr(a_doc, 'words'): a_doc.words = get_doc_words(a_doc)
    for word in a_doc.words: 
        if word in word2id: #проверка для тестового сета
            res[word2id[word]] += 1 
    return res
        
doc2vector(sport_dataset[13])

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
# classifier = RandomForestClassifier(n_estimators=3, max_depth=3)
classifier = RandomForestClassifier(n_estimators=3)

In [42]:
X = []
y = []
for item in sport_dataset:
    X.append(doc2vector(item))
    y.append(1)
    
for item in not_sport_dataset:
    X.append(doc2vector(item))
    y.append(0)

In [43]:
classifier.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
for i, weight in sorted(enumerate(classifier.feature_importances_), key=lambda x: x[1], reverse=True):
    print(i, weight, id2word[i])

103 0.05752976823537722 хороший
405 0.04272600996204552 тренер
499 0.025950247529462348 нокаут
89 0.023851775008523837 матч
506 0.012818218041846591 эстафета
149 0.012174710506108957 го
2025 0.010634469964680175 зенит
396 0.008977775611789869 слуцкий
505 0.008322324233218971 смешанный
2151 0.007932911043317262 болельщик
1248 0.007549278226648915 головин
13 0.00750612780563427 в
465 0.0072945715752016 победа
1254 0.007285895236313401 сезон
1748 0.006973160683117709 сыграть
2021 0.006641145147496437 спартак
488 0.005685088452646143 смотреть
10931 0.005576602109152383 витессать
2386 0.005526898959899255 футбол
84 0.005354885345846019 лига
483 0.00509865483231081 боруссия
136 0.004974684912250534 очки
170 0.004700283911858613 по
413 0.004676240617289493 поражение
509 0.004659783979631001 кубок
3255 0.004618187353058068 реал
81 0.004479825065666455 рубин
74 0.004405193026015545 это
360 0.004301830493079012 dota
20 0.004206978880833995 и
416 0.004073683751709275 сборный
1116 0.00396957767269

23060 0.0001007569500397046 дамба
929 0.00010026098447595305 уголок
6375 0.00010026098447595305 неудача
15880 9.973329508397439e-05 хуслютдин
3009 9.945958435919276e-05 неизвестный
22661 9.945958435919268e-05 нагоя
11434 9.707297763242157e-05 прожиточный
376 9.68461316088536e-05 28
11516 9.625054509691515e-05 krauss
2092 9.604575035588796e-05 80
3441 9.604575035588796e-05 принцип
6891 9.583980308369952e-05 улететь
1176 9.481439458209481e-05 среди
2805 9.458608849637607e-05 депутат
7894 9.457136352931464e-05 звать
2779 9.398930721943691e-05 павел
1369 9.363828585103612e-05 возможность
22234 9.35955898278246e-05 согреться
7730 9.282894540191311e-05 видеотрансляция
7870 9.282894540191311e-05 усман
3414 9.2596219885359e-05 чуть
6660 9.228289278190268e-05 grand
20664 9.214637962689904e-05 204
21941 9.207950997550047e-05 триппьер
125 9.151472479069047e-05 практически
1802 9.102666096233835e-05 подвести
5982 9.049026757668197e-05 магнитогорский
833 8.987140266050105e-05 опубликовать
6292 8.97

765 0.0 айдол
766 0.0 заворожить
767 0.0 декольте
768 0.0 jyp
769 0.0 entertainment
770 0.0 исполнить
771 0.0 хит
772 0.0 don
773 0.0 t
774 0.0 leave
775 0.0 me
776 0.0 1994
777 0.0 you
778 0.0 re
780 0.0 2012
781 0.0 who
782 0.0 s
783 0.0 your
784 0.0 участник
785 0.0 акция
786 0.0 июль
787 0.0 самариддин
788 0.0 раджабов
789 0.0 предъявить
791 0.0 суд
792 0.0 вернуть
793 0.0 несуществующий
794 0.0 преступление
795 0.0 фигурант
797 0.0 правозащита
798 0.0 открытка
799 0.0 ссылка
800 0.0 бадамшин
801 0.0 полимер
802 0.0 кишечный
803 0.0 бактерия
804 0.0 повлиять
805 0.0 восстановление
806 0.0 костный
807 0.0 ткань
809 0.0 мгу
810 0.0 ломоносов
811 0.0 исследовать
812 0.0 полиоксиалканоат
813 0.0 синтезировать
814 0.0 микробиота
815 0.0 соединение
816 0.0 способный
817 0.0 активировать
818 0.0 хрящевой
819 0.0 направление
821 0.0 современный
822 0.0 медицина
823 0.0 позволять
824 0.0 изделие
825 0.0 тканевый
826 0.0 инженерия
828 0.0 поддержать
829 0.0 президентский
831 0.0 научный
834 

2476 0.0 реализовать
2478 0.0 приморье
2479 0.0 продолжаться
2480 0.0 поиск
2481 0.0 судный
2482 0.0 обследовать
2483 0.0 восемь
2485 0.0 береговой
2486 0.0 tanks
2487 0.0 танкист
2488 0.0 снегурочка
2489 0.0 карина
2490 0.0 декабрьский
2491 0.0 уничтожение
2493 0.0 вертолёт
2494 0.0 apache
2495 0.0 зрк
2496 0.0 оса
2497 0.0 сбить
2498 0.0 ah
2499 0.0 64
2500 0.0 предположительно
2501 0.0 инцидент
2502 0.0 аравийский
2503 0.0 полуостров
2504 0.0 уничтожить
2505 0.0 летательный
2506 0.0 принадлежать
2507 0.0 хусито
2508 0.0 обойтись
2510 0.0 зафиксировать
2514 0.0 26
2519 0.0 бабурин
2520 0.0 козлый
2522 0.0 каша
2523 0.0 московск
2524 0.0 поесть
2525 0.0 ресторан
2526 0.0 проверить
2527 0.0 соответствовать
2528 0.0 указанный
2529 0.0 меню
2530 0.0 fifpro
2531 0.0 встать
2533 0.0 конфликт
2534 0.0 международный
2536 0.0 ассоциация
2537 0.0 профессиональный
2538 0.0 санкция
2539 0.0 штраф
2540 0.0 выписать
2543 0.0 бутерброд
2544 0.0 балтийский
2545 0.0 шпрот
2546 0.0 приход
2549 0.0 бан

3895 0.0 курорт
3896 0.0 охт
3898 0.0 гастромаркет
3899 0.0 foods
3900 0.0 in
3901 0.0 woods
3902 0.0 смешать
3903 0.0 азиатский
3904 0.0 кухня
3905 0.0 протест
3906 0.0 прошение
3907 0.0 адель
3908 0.0 абдель
3909 0.0 махдить
3910 0.0 генсек
3911 0.0 напористо
3913 0.0 карманный
3914 0.0 котан
3915 0.0 попросить
3916 0.0 сшить
3917 0.0 крякозябра
3918 0.0 сходство
3919 0.0 феноменальный
3920 0.0 митрополит
3921 0.0 сингапурский
3922 0.0 юго
3923 0.0 сергий
3924 0.0 визит
3925 0.0 литургия
3926 0.0 троицкий
3927 0.0 храм
3928 0.0 нуль
3929 0.0 субботний
3930 0.0 патрульный
3931 0.0 достать
3932 0.0 райан
3933 0.0 реабилитироваться
3934 0.0 джедай
3936 0.0 искромётный
3938 0.0 knives
3939 0.0 out
3940 0.0 райана
3941 0.0 режиссёр
3942 0.0 сценарист
3943 0.0 петлить
3944 0.0 актёрский
3945 0.0 шикарный
3946 0.0 дэниэл
3947 0.0 крэйг
3948 0.0 ана
3949 0.0 армас
3950 0.0 крис
3951 0.0 эванс
3952 0.0 дженять
3953 0.0 кертис
3954 0.0 шеннон
3955 0.0 пламмера
3956 0.0 обескураживать
3957 0.0 

...

25119 0.0 лучо
25120 0.0 здан
25121 0.0 чёртов
25122 0.0 переродиться
25123 0.0 snowпать
25124 0.0 быстрорастущий
25125 0.0 крид
25126 0.0 клава
25127 0.0 st
25128 0.0 zivert
25129 0.0 elvira
25130 0.0 molly
25131 0.0 ternovoy
25132 0.0 artik
25133 0.0 asti
25134 0.0 rsac
25135 0.0 ella
25136 0.0 gayazov
25137 0.0 мерориятие
25138 0.0 dniru
25139 0.0 snowпатм
25140 0.0 обухов
25141 0.0 lgr
25142 0.0 приемлить
25143 0.0 doomd
25144 0.0 флешек
25145 0.0 eternal
25146 0.0 hard
25147 0.0 disk
25148 0.0 sentinel
25149 0.0 ssd
25150 0.0 каждодневно
25151 0.0 lumen
25152 0.0 киберспортивный
25153 0.0 splyce
25154 0.0 mad
25155 0.0 lions
25156 0.0 overactive
25157 0.0 lec
25158 0.0 франшизный
25159 0.0 бобби
25160 0.0 адамс
25161 0.0 виталик
25162 0.0 бутерин
25163 0.0 соучредитель
25164 0.0 криптозащита
25165 0.0 постквантовый
25166 0.0 сброс
25167 0.0 отравлять
25168 0.0 яд
25169 0.0 окись
25170 0.0 этилен
25171 0.0 wildberries
25172 0.0 скрин
25173 0.0 регулярность
25174 0.0 занятой
25175 0

26369 0.0 ругательство
26370 0.0 цзиньпин
26371 0.0 оплевывание
26372 0.0 сынов
26373 0.0 дестабилизировать
26374 0.0 персонально
26375 0.0 высмеивать
26376 0.0 грубость
26377 0.0 макдоналдс
26378 0.0 подливать
26379 0.0 нанимать
26380 0.0 сжигать
26381 0.0 стимулироваться
26382 0.0 обиженный
26383 0.0 возрождаться
26384 0.0 полуколониальный
26385 0.0 типологический
26386 0.0 искусный
26387 0.0 троллинг
26388 0.0 этакий
26389 0.0 слоить
26390 0.0 пиндос
26391 0.0 хамский
26392 0.0 низкопробный
26393 0.0 шовинистический
26394 0.0 барак
26395 0.0 тупо
26396 0.0 похабщина
26397 0.0 различать
26398 0.0 похабник
26399 0.0 скабрёзный
26400 0.0 миросозерцание
26401 0.0 телерадиоканал
26402 0.0 благодушный
26403 0.0 практицизм
26404 0.0 враждебный
26405 0.0 выдрать
26408 0.0 гильотинировать
26409 0.0 усмирять
26410 0.0 надзирать
26411 0.0 ордынка
26412 0.0 dolce
26413 0.0 gabbana
26414 0.0 yana
26415 0.0 rudkovskaya
26416 0.0 rudkovskayaofficial
26417 0.0 няня
26418 0.0 бедроса
26419 0.0 франк

27548 0.0 внешнеэкономический
27549 0.0 просуммировать
27550 0.0 кингисеппский
27551 0.0 спринтер
27552 0.0 сканий
27553 0.0 соцтруд
27554 0.0 минёр
27555 0.0 агапов
27556 0.0 дожить
27557 0.0 трудоспособный
27558 0.0 верняковый
27559 0.0 конопушка
27560 0.0 веснушка
27561 0.0 умилять
27562 0.0 антошка
27563 0.0 лжеполицейский
27564 0.0 непоседливый
27565 0.0 неохотно
27566 0.0 бэкстейдж
27567 0.0 viva
27568 0.0 бордовый
27569 0.0 лео
27570 0.0 подсветить
27571 0.0 стефановский
27572 0.0 осветить
27573 0.0 бант
27574 0.0 ссуза
27575 0.0 безупречно
27576 0.0 элтон
27577 0.0 сим
27578 0.0 нарадоваться
27579 0.0 track
27580 0.0 комаровский
27582 0.0 полярис
27583 0.0 ямало
27584 0.0 многопрофильный
27585 0.0 приветственный
27586 0.0 чигирёв
27587 0.0 завоюр
27588 0.0 субар
27589 0.0 канареечный
27590 0.0 окупанти
27591 0.0 донбасі
27592 0.0 держкордон
27593 0.0 україні
27594 0.0 російський
27595 0.0 ляпас
27596 0.0 проросійські
27597 0.0 бойовик
27598 0.0 донеччині
27599 0.0 відзначиться


28874 0.0 свинцовый
28875 0.0 облёт
28876 0.0 баландино
28877 0.0 локатор
28878 0.0 рлс
28879 0.0 радиозавод
28880 0.0 непрерывность
28881 0.0 аэродромный
28882 0.0 филонец
28883 0.0 пулково
28884 0.0 чрз
28885 0.0 росэлектроника
28886 0.0 нестер
28887 0.0 кфу
28888 0.0 дарвин
28889 0.0 таргет
28890 0.0 таргетировать
28891 0.0 спамить
28892 0.0 тележка
28893 0.0 госпрограмма
28894 0.0 немаркированный
28895 0.0 grotemarktstraat
28896 0.0 кирпиченко
28897 0.0 башкирский
28898 0.0 шестнадцатилетний
28899 0.0 гущина
28900 0.0 свойственно
28901 0.0 окружать
28902 0.0 спонтанно
28903 0.0 майами
28904 0.0 голя
28905 0.0 406060
28906 0.0 поручать
28907 0.0 dior
28908 0.0 круизный
28909 0.0 интерпретация
28910 0.0 намоленість
28911 0.0 евроремонт
28912 0.0 весомый
28913 0.0 кроманьонец
28914 0.0 геноцид
28915 0.0 неандерталец
28916 0.0 homo
28917 0.0 neanderthalensis
28918 0.0 коренастый
28919 0.0 галла
28920 0.0 доко
28921 0.0 футер
28922 0.0 лайкрой
28923 0.0 принтом
28924 0.0 фатина
28925 0.

30205 0.0 кардинал
30206 0.0 лоббировать
30207 0.0 злосчастный
30208 0.0 отсекать
30209 0.0 радикализм
30210 0.0 радушно
30211 0.0 невежество
30212 0.0 межнац
30213 0.0 кэрролл
30214 0.0 дюна
30215 0.0 поспешить
30216 0.0 приспособиться
30217 0.0 выживаемость
30218 0.0 бурковский
30219 0.0 михеев
30220 0.0 алферов
30221 0.0 оренбуржец
30222 0.0 квиза
30223 0.0 мрот
30224 0.0 мотоциклетный
30225 0.0 boxer
30226 0.0 круизер
30227 0.0 оппозитный
30228 0.0 воздуша
30229 0.0 вишнёвый
30230 0.0 дубовиковый
30231 0.0 четырехраундовый
30232 0.0 зав
30233 0.0 malaysia
30234 0.0 airlines
30235 0.0 мясниковский
30236 0.0 экокомплекс
30237 0.0 родство
30238 0.0 докторант
30239 0.0 тартуский
30240 0.0 пеэтера
30241 0.0 аниять
30242 0.0 err
30243 0.0 вулверхэмптон
30245 0.0 джанлука
30246 0.0 марцио
30247 0.0 венеция
30248 0.0 нейробиолог
30249 0.0 поведенческий
30250 0.0 техасский
30251 0.0 прайс
30252 0.0 непереносимый
30253 0.0 офпред
30254 0.0 убогость
30255 0.0 пос
30256 0.0 дооформлять
30257 0

In [45]:
classifier.predict_proba([doc2vector(not_sport_dataset[13])])

array([[1., 0.]])

In [46]:
for item in not_sport_dataset:
    vec = doc2vector(item)
    if classifier.predict([vec]):
        print(item)

url : https://123ru.net/moscow/225922493/
date : 2019-11-30 18:42:20
title : Юлия Оглоблина: «Инкубатор сельских инициатив» стал инновационной платформой для комплексного развития российских сёл
description : Завершает свою работу проект Российского союза сельской молодежи (РССМ) «Инкубатор сельских инициатив». Его целью стало создание, обучение и сопровождение команд развития сельских населенных пунктов. Конкурс на участие в проекте прошли 10 команд из разных уголков России. Под руководством профессионального тьютора и менторов они разрабатывали и реализовывали проекты комплексного развития своих родных территорий. Председатель РССМ, член Центрального штаба ОНФ и зампредседателя комиссии ОП РФ по развитию АПК и сельских территорий Юлия Оглоблина рассказала о проделанной командами работе и планах их работы в следующем году. Читать дальше Источник: «Российский союз сельской молодежи»
site : 123ru.net

url : https://365news.biz/sport/237334-aleksej-melnichuk-s-panarinym-i-kovalchukom-pyt

url : https://russian.rt.com/sport/news/692320-hemilton-kvalifikaciya-abu-dabi
date : 2019-11-30 17:32:41
title : Хэмилтон выиграл квалификацию Гран-при Абу-Даби, Квят — 14-й
description : Британский пилот Mercedes Льюис Хэмилтон одержал победу в квалификации 21-го этапа «Формулы-1» — Гран-при Абу-Даби.
site : russian.rt.com

url : https://wi-fi.ru/news/R5bl1mI391Us-smi-bavariya-i-pszh-proyavlyayut-interes-k-salahu
date : 2019-11-30 17:36:42
title : СМИ: Салахом заинтересовались сразу два клуба
description : "Бавария" и "ПСЖ" рассматривают возможность приобретения нападающего "Ливерпуля" Мохамеда Салаха.
site : wi-fi.ru

url : https://cskanews.com/pfc-cska/news-pfc-cska/milosh-krasich-o-goncharenko-i-sluckom/
date : 2019-11-30 17:38:10
title : Милош Красич о Гончаренко и Слуцком
description : Милош Красич о Гончаренко и Слуцком - Новости ПФК ЦСКА - ЦСКАНЬЮС. ..Бывший футболист ЦСКА Милош Красич поделился мнением о работе главного тренера армейцев Виктора Гончаренко, а также планами на 

url : https://tvzvezda.ru/news/sport/content/201911301424-B7UIh.html
date : 2019-11-30 15:01:40
title : Головин рассказал о феномене Черчесова
description : Игрок сборной России рассказал, в чем секрет Станислава Черчесова при работе с командой. Полузащитник сборной России и французского клуба «Монако» Александр Головин прокомментировал особенности работы главного тренера команды Станислава Черчесова. По его мнению, главный феномен Черчесова заключается в жесткой дисциплине.  «Он от каждого требует показывать на поле максимум: сколько можешь, столько и выкладывайся. Мне кажется, у нас в клубах люди столько не бегают, сколько в сборной», - сказал Головин порталу «Чемпионат». Полузащитник добавил, что Черчесов очень требовательный наставник и с ним «не можешь выйти и недоработать». На должность главного тренера сборной России Станислава Черчесова назначили в 2016 году. В этом году наша сборная под его руководством досрочно вышла в финальную стадию чемпионата Европы-2020, одержав победу н

url : https://ngs55.ru/news/more/66376321/
date : 2019-11-30 13:11:45
title : Братский обмен очками: «Куньлунь» взял реванш у «Авангарда»
description : Матч в Пекине снова завершился не в основное время
site : ngs55.ru

url : http://terrikon.com/posts/341315
date : 2019-11-30 10:29:45
title : Михайленко: Приехали с мыслями о реванше
description : Главный тренер СК «Днепр-1» Дмитрий Михайленко накануне матча против ПФК «Львов» ответил на вопросы клубной пресс-службы
site : terrikon.com

url : https://wi-fi.ru/news/HQmB1xJaC5oV-ovechkin-vyshel-na-tretie-mesto-v-istorii-nhl-po-broskam-za-karieru
date : 2019-11-30 10:49:23
title : Овечкин установил новый ошеломительный рекорд в НХЛ
description : Нападающий "Вашингтона" Александр Овечкин вышел на третье место в истории НХЛ по общему количеству бросков.
site : wi-fi.ru

url : http://terrikon.com/posts/341321
date : 2019-11-30 10:54:10
title : Бавария - Байер: смотреть онлайн-видеотрансляцию Бундеслиги
description : "Бавария" со сменой тренер

In [47]:
#### Тестовые данные
sport_test_dataset = [item for item in dataset_test if item.site in sport_sites]
not_sport_test_dataset = [item for item in dataset_test if item.site not in sport_sites]
print(len(sport_test_dataset), len(not_sport_test_dataset))

391 4609


In [48]:
X_test = []
y_test = []
for item in sport_test_dataset:
    X_test.append(doc2vector(item))
    y_test.append(1)
    
for item in not_sport_test_dataset:
    X_test.append(doc2vector(item))
    y_test.append(0)
    
y_test_pred = classifier.predict(X_test)

In [49]:
import sklearn.metrics

In [50]:
sklearn.metrics.precision_recall_fscore_support(y_test, y_test_pred)

(array([0.95152413, 0.58695652]),
 array([0.97526578, 0.41432225]),
 array([0.96324869, 0.48575712]),
 array([4609,  391]))

In [51]:
sklearn.metrics.confusion_matrix(y_test, y_test_pred)

array([[4495,  114],
       [ 229,  162]])

In [52]:
sklearn.metrics.confusion_matrix?

## Gensim

In [79]:
### https://logic.pdmi.ras.ru/~sergey/slides/N16_DataFestLDA.pdf

In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamulticore
from gensim.models import LdaModel

In [132]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(item.words for item in dataset)

In [133]:
# common_dictionary.filter_extremes()

In [134]:
def show_top_words(a_dictionary, a_top_n=100):
    for i, (id_word, freq) in enumerate(Counter(a_dictionary.dfs).most_common(a_top_n)):
        print(i, a_dictionary[id_word], freq, sep='\t')

In [135]:
show_top_words(common_dictionary)

0	в	7595
1	на	4996
2	и	4793
3	с	3580
4	по	2463
5	не	2273
6	о	2267
7	что	2198
8	из	2004
9	это	1823
10	быть	1762
11	который	1728
12	за	1531
13	год	1527
14	он	1441
15	как	1382
16	для	1362
17	один	1233
18	россия	1182
19	свой	1172
20	к	1119
21	весь	1080
22	от	1076
23	ноябрь	966
24	а	944
25	стать	842
26	они	837
27	российский	820
28	но	815
29	мочь	811
30	у	800
31	тот	794
32	30	791
33	человек	758
34	матч	682
35	время	677
36	новый	668
37	после	657
38	она	653
39	до	648
40	этот	647
41	рассказать	647
42	москва	621
43	два	612
44	1	598
45	самый	578
46	сообщать	556
47	при	554
48	хороший	549
49	мы	542
50	такой	529
51	область	518
52	2	511
53	так	508
54	день	505
55	то	493
56	уже	493
57	мир	493
58	заявить	482
59	декабрь	477
60	под	463
61	главный	454
62	также	453
63	2019	448
64	сообщить	447
65	ещё	431
66	сайт	430
67	место	428
68	большой	427
69	только	421
70	или	414
71	команда	410
72	видео	410
73	александр	410
74	можно	407
75	я	405
76	район	396
77	страна	394
78	пройти	390
79	какой	388
80	2020	388
81	чтобы	

In [136]:
# common_dictionary.filter_n_most_frequent(17)

In [137]:
def filter_short_words(words):
    for word in words:
        if len(word) >= 3: yield word

In [138]:
common_dictionary = Dictionary(filter_short_words(item.words) for item in dataset)

In [139]:
show_top_words(common_dictionary)

0	что	2198
1	это	1823
2	быть	1762
3	который	1728
4	год	1527
5	как	1382
6	для	1362
7	один	1233
8	россия	1182
9	свой	1172
10	весь	1080
11	ноябрь	966
12	стать	842
13	они	837
14	российский	820
15	мочь	811
16	тот	794
17	человек	758
18	матч	682
19	время	677
20	новый	668
21	после	657
22	она	653
23	этот	647
24	рассказать	647
25	москва	621
26	два	612
27	самый	578
28	сообщать	556
29	при	554
30	хороший	549
31	такой	529
32	область	518
33	так	508
34	день	505
35	уже	493
36	мир	493
37	заявить	482
38	декабрь	477
39	под	463
40	главный	454
41	также	453
42	2019	448
43	сообщить	447
44	ещё	431
45	сайт	430
46	место	428
47	большой	427
48	только	421
49	или	414
50	команда	410
51	видео	410
52	александр	410
53	можно	407
54	район	396
55	страна	394
56	пройти	390
57	какой	388
58	2020	388
59	чтобы	386
60	если	383
61	дело	379
62	сегодня	373
63	другой	366
64	работа	360
65	город	358
66	получить	355
67	дом	354
68	слово	350
69	президент	342
70	летний	339
71	назвать	339
72	появиться	339
73	новость	338
74	провести	336
75	п

In [140]:
common_dictionary.filter_n_most_frequent(7)

In [141]:
common_corpus = [common_dictionary.doc2bow(item.words) for item in dataset]

In [146]:
# Train the model on the corpus.
# lda = ldamulticore.LdaMulticore(common_corpus, num_topics=10, workers=1)
lda = LdaModel(common_corpus, num_topics=100, id2word=common_dictionary)

In [147]:
lda.show_topics(50)

[(0,
  '0.009*"москва" + 0.008*"район" + 0.008*"ноябрь" + 0.008*"декабрь" + 0.007*"произойти" + 0.006*"автомобиль" + 0.006*"россия" + 0.006*"один" + 0.005*"дтп" + 0.005*"город"'),
 (1,
  '0.009*"свой" + 0.007*"новый" + 0.006*"летний" + 0.005*"она" + 0.005*"компания" + 0.005*"девочка" + 0.005*"человек" + 0.004*"учёный" + 0.004*"стать" + 0.004*"весь"'),
 (2,
  '0.013*"тренер" + 0.013*"слуцкий" + 0.011*"мир" + 0.009*"российский" + 0.008*"россия" + 0.008*"главный" + 0.008*"этап" + 0.007*"кубок" + 0.007*"уфа" + 0.007*"спартак"'),
 (3,
  '0.008*"полиция" + 0.007*"2020" + 0.006*"один" + 0.006*"два" + 0.005*"ноябрь" + 0.005*"россия" + 0.005*"медведев" + 0.005*"ван" + 0.005*"отделение" + 0.004*"лондонский"'),
 (4,
  '0.011*"украина" + 0.008*"россия" + 0.006*"сша" + 0.006*"ядерный" + 0.005*"мочь" + 0.005*"война" + 0.005*"новый" + 0.005*"украинский" + 0.004*"свой" + 0.004*"область"'),
 (5,
  '0.014*"хороший" + 0.009*"момент" + 0.007*"они" + 0.007*"свой" + 0.007*"видео" + 0.006*"весь" + 0.005*"мат

In [153]:
# Compute Perplexity
print('Perplexity: ', lda.log_perplexity(common_corpus))  # a measure of how good the model is. lower the better.

Perplexity:  -9.66079296235385


In [148]:
doc_lda = lda[common_corpus]

In [149]:
for i,item in itertools.islice(enumerate(doc_lda), 100):
    print(item)
    print(dataset[i])

[(4, 0.82652164), (6, 0.11182616)]
url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

[(5, 0.9804564)]
url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами были установле(2)ы часы, а рядом с ними […]
site : trikky.ru

[(1, 0.9334135)]
url : https://topwar.ru/165315-chernomorskij-flot-popolnilsja-150-tonnym-plavkranom-proekta-02690.html
date : 2019-11-30 18

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# # See trigram example
# print(trigram_mod[bigram_mod[data_words[0]]])

In [158]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, common_corpus, common_dictionary)
vis

/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.081502  0.120034       1        1  23.484024
6     -0.024501 -0.085131       2        1  12.370799
0     -0.080180 -0.164112       3        1  11.243063
8      0.108616  0.002361       4        1   9.964696
1     -0.095383  0.026361       5        1   9.040228
5     -0.049082  0.107257       6        1   8.766610
4      0.053144  0.020904       7        1   7.526371
9     -0.032011  0.051471       8        1   6.468295
2      0.201740  0.012494       9        1   6.293571
3     -0.000842 -0.091640      10        1   4.842347, topic_info=    Category         Freq     Term        Total  loglift  logprob
70   Default   838.000000     матч   838.000000  30.0000  30.0000
113  Default   738.000000  хороший   738.000000  29.0000  29.0000
121  Default  1726.000000   россия  1726.000000  28.0000  28.0000
360  Default   420.000000   тренер   420.000000  27.0000  27.0000
359  Default   278.000000  слуцкий   278.000000  26.0000  26.0000
..       ...          ...      ...          ...      ...      ...
200  Topic10    54.915989   пройти   449.145660   0.9262  -5.6791
696  Topic10    50.506538  мужчина   462.851654   0.8125  -5.7629
357  Topic10    48.649181     свой  1708.763794  -0.5311  -5.8003
31   Topic10    46.140194      они  1352.657593  -0.3504  -5.8533
163  Topic10    45.050426    стать  1017.902832  -0.0899  -5.8772

[787 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1697      8  0.978346     120
7416      6  0.965239     128
4262      2  0.096542     200
4262      4  0.096542     200
4262      5  0.717166     200
...     ...       ...     ...
405       5  0.008247  япония
405       7  0.651548  япония
405       8  0.008247  япония
405       9  0.041237  япония
405      10  0.065980  япония

[2873 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 1, 9, 2, 6, 5, 10, 3, 4])

In [159]:
from gensim.models import word2vec

In [182]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset_w2v = []
for line in itertools.islice(fin, 100000):
    data = json.loads(line.strip())
    dataset_w2v.append(Document(data))

In [183]:
%%time
for item in dataset_w2v: item.words = list(get_doc_words(item))

CPU times: user 12min 50s, sys: 16.4 s, total: 13min 6s
Wall time: 13min 25s


In [184]:
w2v = word2vec.Word2Vec([item.words for item in dataset_w2v], workers=4)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [188]:
w2v.most_similar('bmw') #

/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hyundai', 0.8927821516990662),
 ('volkswagen', 0.8895480036735535),
 ('nissan', 0.888046383857727),
 ('toyota', 0.8622678518295288),
 ('benz', 0.8618403673171997),
 ('купе', 0.8612299561500549),
 ('avant', 0.8603546023368835),
 ('mitsubishi', 0.8601386547088623),
 ('rs', 0.859824538230896),
 ('audi', 0.859603762626648)]

In [189]:
w2v['bmw']

/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.02018423, -1.0606977 ,  0.21468312, -0.6280541 ,  0.16023648,
       -0.09056458, -0.21388392, -0.3693361 ,  0.5253866 , -0.76814675,
        0.7476715 , -0.30161467, -1.2888315 ,  1.2124308 ,  0.32328054,
        0.08970812,  0.0253071 ,  0.94276446, -0.526752  ,  0.8245327 ,
        0.89138335,  0.589797  ,  0.07530585, -1.3981738 ,  0.0210677 ,
       -1.322176  , -1.159928  , -0.7588753 , -0.48051867, -1.2741979 ,
        0.9480522 , -0.01111534, -1.0424693 , -0.13723288,  0.0294291 ,
       -0.08984844,  0.44023997,  2.120343  ,  1.2159361 , -0.42526478,
       -1.0878861 , -1.0340353 , -0.90502673,  0.33292717,  0.65223384,
       -0.8889226 ,  0.73234475, -0.08602537, -1.2959174 ,  0.35444507,
       -0.9630726 , -1.5606074 , -1.217541  ,  0.83545834, -1.299714  ,
        0.2570357 , -0.0237768 , -0.12570556, -0.18834434,  0.31774476,
        0.05251185,  0.19539082,  1.3707831 ,  0.38835183,  0.45673037,
        1.168465  ,  0.80192167,  0.3609308 ,  0.56230813,  1.06

In [190]:
from scipy.spatial.distance import cosine

In [195]:
cosine(w2v['bmw'], w2v['bmw'])

/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


0.0

In [196]:
cosine(w2v['bmw'], w2v['mercedes'])

/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/Users/a.murashev/miniconda2/envs/py37_mai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


0.19826781749725342

## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам

In [12]:
def doc2vec(a_doc, a_w2v):
    res = None
    for word in a_doc.words:
        if word not in a_w2v.vocabulary:
            continue
        if res is None: 
            res = a_w2v[word]
        else:
            res += a_w2v[word]
    return res

In [8]:
def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=20):
    pass

def get_tf_idf_most_similar_doc(a_doc, dataset):
    # для каждого документа строится вектор размерности словаря (аналогично random forest) и добавляется idf
    # далее cosine между документами
    pass

def get_w2v_most_similar_doc(a_doc, dataset):
    # считается средний вектор по всем словам (можно при усреднении учитывать idf)
    pass

In [9]:
doc_id = 13
for sim_func in [get_word_match_most_similar_docs, get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    sim_func(dataset[doc_id], dataset)

NameError: name 'dataset' is not defined